In [1]:
import torch
from torch_geometric.datasets import MoleculeNet
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
from torch_geometric.utils.convert import to_networkx
from networkx import all_pairs_shortest_path

In [2]:
!pip install rdkit

In [3]:
dataset = MoleculeNet(root="./", name="ESOL")
dataset

ESOL(1128)

In [4]:
from torch import nn
import torch_geometric.nn as tgnn
from graphormer.model import Graphormer


model = Graphormer(
    num_layers=3,
    input_node_dim=dataset.num_node_features,
    node_dim=16,
    input_edge_dim=dataset.num_edge_features,
    edge_dim=16,
    output_dim=dataset[0].y.shape[1],
    n_heads=4,
    max_in_degree=5,
    max_out_degree=5,
    max_path_distance=5,
)

In [5]:
loader = DataLoader(dataset, batch_size=16)

In [6]:
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)
loss_functin = nn.L1Loss(reduction="sum")

In [ ]:
from tqdm import tqdm
from torch_geometric.nn.pool import global_mean_pool


model.train()
for epoch in range(10):
    batch_loss = 0.0
    for batch in tqdm(loader):
        y = batch.y
        optimizer.zero_grad()
        output = global_mean_pool(model(batch), batch.batch)
        loss = loss_functin(output, y)
        batch_loss += loss.item()
        loss.backward()
        optimizer.step()
    
    print(total_loss)

  0%|                                                                                                                                                | 0/71 [00:00<?, ?it/s]